## Initialize Fiddler Client


In [5]:
import fiddler as fdl
client = fdl.FiddlerApi('https://staging1.fiddler.ai', 'manojtestorg', 'cQTLmdzxQmnym5C3sEJfRA74s9SiCcaqytNSjUXFNAL')

## Load baseline

In [6]:
import pandas as pd
baseline_df = pd.read_csv('/app/fiddler_samples/samples/datasets/p2p_loans/p2p_loans.csv')
baseline_schema = fdl.DatasetInfo.from_dataframe(baseline_df, max_inferred_cardinality=1000)

## Setup monitoring

In [7]:

project_id = 'tutorial'
model_id = 'loan_status_surrogate'
target='loan_status'
features = ['loan_amnt', 'int_rate', 'annual_inc', 'dti', 'fico_range_low', 'total_acc', 'acc_open_past_24mths']

## setup/cleanup project
if project_id in client.list_projects():
    client.delete_model(project_id, model_id)
    client.delete_dataset(model_id)
else:
    client.create_project(project_id)


client.create_surrogate_model(
    project_id,
    model_id,
    baseline_df,
    baseline_schema,
    target,
    features
)


Validating inputs ...
Uploading dataset ...
Generating surrogate model ...
Triggering model predictions ...


'Surrogate model successfully setup on Fiddler. \n Visit https://staging1.fiddler.ai/projects/tutorial'

## Send monitoring events

In [ ]:
import datetime
import random
from IPython.display import clear_output
import time

two_weeks_ms = 604800 * 2 * 1000

result_df = pd.read_csv('/app/fiddler_samples/samples/datasets/p2p_loans/p2p_production.log')
chunked_df = [result_df[i:i+1] for i in range(0, result_df.shape[0], 1)]
count = 0
for row_df in chunked_df:
    data = row_df.to_dict('records')[0]
    occurred_at = int(round(time.time() * 1000)) - random.randint(0, two_weeks_ms)
    
    result = client.publish_event(project_id, model_id, data, event_time_stamp=occurred_at)
    
    count += 1
    clear_output(wait = True)
    print(f'Sending {count} / {len(result_df)}')
    time.sleep(2)

Sending 71 / 2997
